In [62]:
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import math
import random
from decimal import Decimal, getcontext
import pandas_market_calendars as mcal
import ast
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import pytz

In [63]:
s3 = boto3.client('s3')

# Create a calendar
nyse = mcal.get_calendar('NYSE')
holidays = nyse.regular_holidays
market_holidays = holidays.holidays()

index = ["QQQ","SPY","IWM","TLT"]
test_lag = ["DAL","UAL","VZ","T","AAL","AMC"]
train_lag = ["AMC"]
bf_plus = ["AMD","NVDA","PYPL","GOOG","GOOGL","AMZN","PLTR","BAC","AAPL","NFLX","ABNB","CRWD","SHOP","FB","CRM",
            "MSFT","F","V","MA","JNJ","DIS","JPM","INTC","ADBE","BA","CVX","MRNA","PFE","SNOW","NKE",'META',
            'C','TGT','MMM','SQ','PANW','DAL','CSCO','UBER','SBUX']
bfpidx = ["AMD","NVDA","PYPL","GOOG","GOOGL","AMZN","PLTR","BAC","AAPL","NFLX","ABNB","CRWD","SHOP","FB","CRM",
            "MSFT","F","V","MA","JNJ","DIS","JPM","INTC","ADBE","BA","CVX","MRNA","PFE","SNOW","NKE",'META',
            'C','TGT','MMM','SQ','PANW','DAL','CSCO','UBER','SBUX','TSM',"QQQ","SPY","IWM","TLT"]

high_vol = ['ZM', 'UBER', 'TDOC', 'UAL', 'DAL', 'RCL', 'AMZN', 'ABNB', 'CRM',
       'F', 'ADBE', 'BA', 'META', 'TSLA', 'LCID', 'NIO', 'RIVN', 'COIN',
       'SQ', 'SHOP', 'DOCU', 'ROKU', 'TWLO', 'DDOG', 'ZS', 'NET', 'OKTA',
       'UPST', 'ETSY', 'PINS', 'FUTU', 'SE', 'BIDU', 'JD', 'BABA', 'RBLX',
       'AMD', 'NVDA', 'PYPL', 'PLTR', 'NFLX', 'CRWD', 'INTC', 'MRNA',
       'SNOW', 'SOFI', 'PANW']

low_vol = ['CMG', 'AXP', 'MMM', 'PEP', 'GE', 'MRK', 'HD', 'LOW', 'VZ', 'PG',
            'TSM', 'GOOG', 'GOOGL', 'BAC', 'AAPL', 'MSFT', 'V', 'MA', 'JNJ',
            'DIS', 'JPM', 'CVX', 'PFE', 'C', 'CAT', 'KO', 'MS', 'GS', 'IBM',
            'CSCO', 'WMT', 'WFC', 'TGT', 'COST', 'ORCL', 'SBUX', 'NKE', 'QQQ',
            'SPY', 'TLT', 'IWM']

goliath = ['QQQ','SPY','NVDA','IWM','TLT','AAPL','MSFT','GOOG','GOOGL','AMZN','META','FB']

label_encoder = LabelEncoder()


In [64]:
def create_training_data_local(key_list, prefix, bucket_name, start_date, end_date):
    df_list = []
    hours = [10,11,12,13,14,15]
    start = start_date.split(' ')[0]
    end = end_date.split(' ')[0]
    # print(file_key)
    for key in key_list:
        for hour in hours:
            try:
                data = s3.get_object(Bucket=bucket_name, Key=f'{prefix}{key}/{hour}.csv')
                df = pd.read_csv(data.get("Body")) 
                df_list.append(df)
                # df['hour'] = hour
            except:
                continue

    data = pd.concat(df_list)
    data = data.loc[~data['symbol'].isin(index)]
    # data = data.loc[~data['symbol'].isin(laggards)]
    data.reset_index(drop=True, inplace=True)
    # data['date'] = pd.to_datetime(data['date'])
    # data['day_of_week'] = data['dt'].apply(lambda x: x.dayofweek)
    # data = data.round(3)
    data.replace([np.inf, -np.inf], 0, inplace=True)

    data.to_csv(f'/Users/charlesmiller/Documents/BF/{start}_{end}.csv', index=False)

    return data

def pull_training_data_local(end_date,start_date):
    data = pd.read_csv(f'/Users/charlesmiller/Documents/ALL_SYM/2018-01-01_2023-10-28.csv')
    data['dt'] = pd.to_datetime(data['date'])
    data = data.loc[data['dt'] <= end_date]
    data = data.loc[data['symbol'].isin(goliath)]
    data['symbol_encoded'] = label_encoder.fit_transform(data['symbol'])
    data.replace([np.inf, -np.inf], 0, inplace=True)

    return data


def pull_validation_data_local(end_date,start_date):
    data = pd.read_csv(f'/Users/charlesmiller/Documents/ALL_SYM/2018-01-01_2023-10-28.csv')
    data['dt'] = pd.to_datetime(data['date'])
    data = data.loc[data['dt'] <= end_date]
    data = data.loc[data['dt'] >= start_date]
    data = data.loc[data['symbol'].isin(goliath)]
    data['symbol_encoded'] = label_encoder.fit_transform(data['symbol'])
    data.replace([np.inf, -np.inf], 0, inplace=True)

    return data

In [65]:
def build_date_list(start_date, end_date):
    print(start_date, end_date)
    date_diff = end_date - start_date
    numdays = date_diff.days 
    dateList = []
    for x in range (0, numdays):
        temp_date = start_date + timedelta(days = x)
        if temp_date.weekday() > 4:
            continue
        else:
            dateList.append(temp_date)
    return dateList

def build_query_keys_hist():
    start_date = datetime(2021,1,5)
    date_diff = datetime(2022,7,29) - start_date
    numdays = date_diff.days 
    key_list = []
    for x in range (0, numdays):
        temp_date = start_date + timedelta(days = x)
        if temp_date.weekday() > 4:
            continue
        else:
            date_str = temp_date.strftime('%Y-%m-%d')
            if date_str in market_holidays:
                continue
            else:
                date_str = date_str.replace("-","/")
                key_list.append(date_str)
        
    return key_list
    
def build_query_keys(dates):
    key_list = []
    for date in dates:
        date_str = date.strftime('%Y-%m-%d')
        if date_str in market_holidays:
            continue
        else:
            year, month, day = date_str.split('-')
            temp = f'{year}/{month}/{day}'
            key_list.append(temp)

    return key_list

def build_query_keys_validation(end_date):
    validation_end_date = end_date + timedelta(days=7)
    dates = build_date_list(end_date, validation_end_date)
    key_list = []
    for date in dates:
        date_str = date.strftime('%Y-%m-%d')
        if date_str in market_holidays:
            continue
        else:
            year, month, day = date_str.split('-')
            temp = f'{year}/{month}/{day}'
            key_list.append(temp)

    return key_list

def build_validation_dates_local(deployment_date):
    end_date = deployment_date + timedelta(days=5)
    return end_date

In [66]:
def quantile_loss_np(y_true, y_pred, quantile):
    """
    Compute the quantile loss using NumPy for a specific quantile.

    :param y_true: NumPy array of true values
    :param y_pred: NumPy array of predicted values
    :param quantile: The quantile to be estimated (e.g., 0.1, 0.5, 0.9)
    :return: Quantile loss
    """
    e = y_true - y_pred
    return np.mean(np.maximum(quantile * e, (quantile - 1) * e))

def model_results_analyzer(predictions, y_validate):
    quantile = quantile_loss_np(y_validate, predictions, quantile=0.5)
    mse = mean_squared_error(y_validate, predictions)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(y_validate, predictions)
    r2 = r2_score(y_validate, predictions)
    print(f'MSE: {mse}')
    print(f'Pinball Loss: {quantile}')
    return mse, rmse, mae, r2, quantile

In [67]:
def create_dynamo_record(tp, tp_scr, fp, fp_scr, tn, tn_scr, fn, fn_scr, model_name, deployment_date, dataset_name, hyperparam_str, feature_str, target_str, fi_list):    
    ddb = boto3.resource('dynamodb','us-east-1')
    table = ddb.Table('icarus-models-results-table')
    eval_start = deployment_date
    eval_end = deployment_date + timedelta(days=4)
    try:
        precision = (tp/fp)
    except:
        precision = 0

    ## FILL IN
    item={
        'model_name': model_name,
        'deployment_date': deployment_date.strftime("%Y-%m-%d"),
        'algorithm_type': 'xgboost',
        'dataset': dataset_name,
        'TP': tp,
        'TPpct': Decimal("0"),
        'FP': fp,
        'FPpct': Decimal("0"),
        'TN': tn,
        'TNpct': Decimal("0"),
        'FN': fn,
        'FNpct': Decimal("0"),
        'precision_ratio': Decimal(str(precision)),
        'evaluation_timeperiod': f'{eval_start.strftime("%Y-%m-%d")}_{eval_end.strftime("%Y-%m-%d")}',
        'live': False,
        'hyperparameters': {hyperparam_str},
        'features' : {feature_str},
        'target' : target_str,
        'feature_importances': fi_list

    }

    print(item)
    response = table.put_item(
            Item=item
        )

    return response

In [68]:
def train_model(features, dataset, validation_dataset, target_label, target_value, hyperparams):
    X = dataset[features]
    y = dataset[target_label]

    X_validate = validation_dataset[features]
    y_validate = validation_dataset[target_label]

    # X['symbol_encoded'] = X['symbol_encoded'].astype('category')
    # X_validate['symbol_encoded'] = X_validate['symbol_encoded'].astype('category')
    # dtrain = xgb.DMatrix(X, label=y, enable_categorical=True)
    # dtest = xgb.DMatrix(X_validate, label=y_validate, enable_categorical=True)


    # params = {"subsample":hyperparams['subsample'],"num_round":hyperparams['num_round'],"min_child_weight":hyperparams['min_child_weight'],"max_depth":hyperparams['max_depth'],"learning_rate":hyperparams['learning_rate'],"gamma":hyperparams['gamma'],"colsample_bytree":hyperparams['colsample_bytree'],"verbosity":0,"objective":'reg:squarederror',"random_state":42}
    # xgb_model = xgb.train(params, dtrain)

    xgb_model = xgb.XGBRegressor(alpha=0.5,subsample=hyperparams['subsample'],num_round=hyperparams['num_round'],min_child_weight=hyperparams['min_child_weight'],max_depth=hyperparams['max_depth'],learning_rate=hyperparams['learning_rate'],gamma=hyperparams['gamma'],colsample_bytree=hyperparams['colsample_bytree'],verbosity=0,objective='reg:linear',random_state=42)
    xgb_model.fit(X,y)

    predictions = xgb_model.predict(X_validate)
    mse, rmse, mae, r2, quantile_loss = model_results_analyzer(predictions, y_validate)
    validation_dataset['forecast'] = predictions


    fi = xgb_model.feature_importances_
    fi_list = []
    counter = 0
    for x in features:
        fi_list.append({x:fi[counter]})
        counter += 1
    return mse, rmse, mae, r2, fi_list, validation_dataset, quantile_loss


def train_model_TSSim(features, dataset, validation_dataset, target_label, target_value, hyperparams):
    dataset = dataset.round(3)
    validation_dataset = validation_dataset.round(3)


    X = dataset[features].astype(float)
    y = dataset[target_label]

    X_validate = validation_dataset[features].astype(float)
    y_validate = validation_dataset[target_label]
    

    xgb_model = xgb.XGBRegressor(subsample=hyperparams['subsample'],num_round=hyperparams['num_round'],min_child_weight=hyperparams['min_child_weight'],max_depth=hyperparams['max_depth'],learning_rate=hyperparams['learning_rate'],gamma=hyperparams['gamma'],colsample_bytree=hyperparams['colsample_bytree'],verbosity=0,objective='reg:squarederror',random_state=42)
    xgb_model.fit(X,y)

    predictions = xgb_model.predict(X_validate)
    mse, rmse, mae, r2 = model_results_analyzer(predictions, y_validate)
    validation_dataset['forecast'] = predictions

    return mse, rmse, mae, r2, "fi_list", validation_dataset

In [69]:
def model_runner_v2(model_name, dataset_name, title, features, target_label, target_value,start_date, end_date,deployment_date, feature_str, hyperparams_str, hyperparams,local_data, dataset_start_date):
    validation_end_date = build_validation_dates_local(deployment_date)
    validation_dataset = pull_validation_data_local(validation_end_date,deployment_date)
    dataset = pull_training_data_local(end_date, dataset_start_date)
    dataset.dropna(subset=["close_diff_deviation3"],inplace=True)
    validation_dataset.dropna(subset=["close_diff_deviation3"],inplace=True)
    mse, rmse, mae, r2, fi_list, validation_dataset, quantile_loss = train_model(features, dataset, validation_dataset, target_label, target_value, hyperparams)
    put_response = s3.put_object(Bucket="icarus-research-data", Key=f"backtesting_data/inv_alerts/{dataset_name}/{title}/{deployment_date.strftime('%Y-%m-%d')}.csv", Body=validation_dataset.to_csv())
    return put_response

def model_runner_temporal_simulation(features, target_label, target_value,dataset_start_date,end_date,deployment_date,hyperparams):
    validation_end_date = build_validation_dates_local(deployment_date)
    dataset = pull_training_data_local(end_date, dataset_start_date)
    validation_dataset = pull_validation_data_local(validation_end_date,deployment_date)
    dataset.dropna(subset=["close_diff_deviation3"],inplace=True)
    validation_dataset.dropna(subset=["close_diff_deviation3"],inplace=True)
    dataset.dropna(subset=["close_diff_deviation"],inplace=True)
    validation_dataset.dropna(subset=["close_diff_deviation"],inplace=True)
    mse, rmse, mae, r2, fi_list, validation_dataset = train_model_TSSim(features, dataset, validation_dataset, target_label, target_value, hyperparams)
    return mse

def model_runner_data(start_date,end_date,):
    dates = build_date_list(start_date, end_date)
    key_list = build_query_keys(dates)
    print(key_list[-1])
    dataset = create_training_data_local(key_list, 'bf/vol/', 'inv-alerts', start_date.strftime('%Y-%m-%d %H:%M:%S'),end_date.strftime('%Y-%m-%d %H:%M:%S'))
    return dataset

In [70]:
def build_evaluation_period(eval_start, eval_end):
    dates_list = []
    while eval_start <= eval_end:
        date_object = {
            "deployment_date": eval_start,
            "dataset_end": eval_start - timedelta(days=10),
            "dataset_start": datetime(2018,1,1)
        }
        dates_list.append(date_object)
        eval_start += timedelta(days=7)
    return dates_list

In [71]:
model_name = 'BFC:TSSIM1_GOLIATHquant_custHyp_2018'
dataset_name = 'TSSIM1_GOLIATHquant_custHyp_2018'
title = 'BFC'
hyperparams = {'subsample': 0.6, 'num_round': 1000, 'min_child_weight': 10, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 2, 'colsample_bytree': 1}
features = ['min_volume_vol_diff', 'daily_vol_diff', 'volume_vol_450M', 'threeD_stddev50', 'return_vol_15M', 'rsi', '15min_vol_diff_pct', 'return_vol_4H', 
             'volume_vol_30M', 'SPY_3D', 'SPY_diff', 'volume_vol_60M', 'SPY_5D', 'oneD_stddev50', 'rsi3', 'roc', 'adx', 'SPY_diff3', 'volume_vol_240M', 'hour_vol_diff', 
             'cmf', 'month', 'volume_vol_16H', 'daily_volume_vol_diff_pct', 'close_diff3', 'return_vol_240M', 'hour', 'return_vol_3D', 'volume_vol_4H', 'range_vol',
             'volume_vol_8H', 'daily_vol_diff_pct30', 'volume_vol_15M', 'threeD_returns_close']
target_label = 'three_max'
target_value = .025

dataset_start_date = datetime(2018,1,1,tzinfo=pytz.timezone('US/Eastern')) 
dates_list = build_evaluation_period(datetime(2022,10,3,tzinfo=pytz.timezone('US/Eastern')), datetime(2023,10,14,tzinfo=pytz.timezone('US/Eastern')))

# last_date = dates_list[-1]
# print(date)
# response = model_runner_data(start_date=datetime(2023,10,11),end_date=datetime(2023,10,28))
for date in dates_list:
    print(date)
    response = model_runner_v2(model_name, dataset_name, title, features, target_label, target_value,"dataset_start_date",date['dataset_end'],date['deployment_date'], str(features),str(hyperparams), hyperparams, local_data=True, dataset_start_date=date['dataset_start'])
    


{'deployment_date': datetime.datetime(2022, 10, 3, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>), 'dataset_end': datetime.datetime(2022, 9, 23, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>), 'dataset_start': datetime.datetime(2018, 1, 1, 0, 0)}
MSE: 0.00041204410873640554
Pinball Loss: 0.008818398993439747
{'deployment_date': datetime.datetime(2022, 10, 10, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>), 'dataset_end': datetime.datetime(2022, 9, 30, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>), 'dataset_start': datetime.datetime(2018, 1, 1, 0, 0)}
MSE: 0.000735602602417089
Pinball Loss: 0.010069326052069663
{'deployment_date': datetime.datetime(2022, 10, 17, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>), 'dataset_end': datetime.datetime(2022, 10, 7, 0, 0, tzinfo=<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>), 'dataset_start': datetime.datetime(2018, 1, 1, 0, 0)}
MSE: 0.0007572768673217998
Pinball Lo

In [72]:
#### LOCAL DATA CREATION

In [73]:
# data = pd.read_csv(f'/Users/charlesmiller/Documents/BF/2018-01-01_2023-10-07.csv')
# # data['date'] = data['date_x'].astype(str)
# # data['date'] = data['date_x'].apply(lambda x: x.split(" ")[0])
# data['date'] = pd.to_datetime(data['date'])
# # data.drop(['one_pct','three_pct','Unnamed: 0.2', "Unnamed: 0","Unnamed: 0.1", "date_x","date_y",'vw_x', 'n_x', 'return_vol_240M_x', 'volume_vol_240M_x', 'return_vol_450M_x', 'volume_vol_450M_x', 'min_vol_diff_x', 'min_vol_diff_pct_x', 'min_volume_vol_diff_x', 'min_volume_vol_diff_pct_x', 'return_vol_8H_x', 'return_vol_16H_x', 'volume_vol_8H_x', 'volume_vol_16H_x', 'hour_vol_diff_x', 'hour_vol_diff_pct_x', 'hour_volume_vol_diff_x', 'hour_volume_vol_diff_pct_x', 'return_vol_5D_x', 'return_vol_10D_x', 'return_vol_30D_x', 'volume_vol_5D_x', 'volume_vol_10D_x', 'volume_vol_30D_x', 'daily_vol_diff_x', 'daily_vol_diff_pct_x', 'daily_vol_diff30_x', 'daily_vol_diff_pct30_x', 'daily_volume_vol_diff_x', 'daily_volume_vol_diff_pct_x', 'daily_volume_vol_diff30_x', 'daily_volume_vol_diff_pct30_x', 'vw_y', 'n_y', 'return_vol_240M_y', 'volume_vol_240M_y', 'return_vol_450M_y', 'volume_vol_450M_y', 'min_vol_diff_y', 'min_vol_diff_pct_y', 'min_volume_vol_diff_y', 'min_volume_vol_diff_pct_y', 'return_vol_8H_y', 'return_vol_16H_y', 'volume_vol_8H_y', 'volume_vol_16H_y', 'hour_vol_diff_y', 'hour_vol_diff_pct_y', 'hour_volume_vol_diff_y', 'hour_volume_vol_diff_pct_y', 'return_vol_5D_y', 'return_vol_10D_y', 'return_vol_30D_y', 'volume_vol_5D_y', 'volume_vol_10D_y', 'volume_vol_30D_y', 'daily_vol_diff_y', 'daily_vol_diff_pct_y', 'daily_vol_diff30_y', 'daily_vol_diff_pct30_y', 'daily_volume_vol_diff_y', 'daily_volume_vol_diff_pct_y', 'daily_volume_vol_diff30_y', 'daily_volume_vol_diff_pct30_y'], axis=1, inplace=True)
# data['roc_diff'] = data['roc'] - data['roc5']
# data['range_vol_diff5'] = (data['range_vol'] - data['range_vol5MA'])
# data['close_diff_deviation3'] = abs(data['close_diff3'])/(data['threeD_stddev50']*100)
# data['close_diff_deviation'] = abs(data['close_diff'])/(data['oneD_stddev50']*100)
# data.dropna(subset=['date'], inplace=True)
# data['day_of_week'] = data['date'].apply(lambda x: x.dayofweek).astype(int)
# data['day_of_month'] = data['date'].apply(lambda x: x.day).astype(int)
# data['month'] = data['date'].apply(lambda x: x.month).astype(int)
# data['year'] = data['date'].apply(lambda x: x.year).astype(int)
# data.replace([np.inf, -np.inf], 0, inplace=True)
# data.to_csv(f'/Users/charlesmiller/Documents/BF/2018-01-01_2023-10-07.csv', index=False)

In [74]:
# def run_temporal_simulation(features, target_label, target_value, dataset_start_date, evaluation_start_date, evaluation_end_date):
#     mse_list = []
    
#     hyperparams = {'subsample': 0.6, 'num_round': 1000, 'min_child_weight': 10, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 2, 'colsample_bytree': 1}
#     dates_list = build_evaluation_period(evaluation_start_date, evaluation_end_date)
#     # date = dates_list[-1]
#     # response = model_runner_data(model_name, dataset_name, title, features, target_label, target_value,dataset_start_date,date['dataset_end'],date['deployment_date'], str(features),str(hyperparams), hyperparams)
#     for date in dates_list:
#         mse = model_runner_temporal_simulation(features, target_label, target_value,dataset_start_date,date['dataset_end'],date['deployment_date'], hyperparams)
#         mse_list.append(mse)

#     return mse_list

In [75]:
# number_of_simulations = 15
# now = datetime.now()
# title = 'BFC3d_reg'
# total_feature_list = ['hour','year','month','day_of_month',
#        'price7', 'price14', 'adjusted_volume', 'vol7', 'vol14',
#        'rsi', 'rsi3', 'rsi5', 'roc', 'roc3', 'roc5', 'threeD_returns_close',
#        'oneD_returns_close', 'range_vol', 'range_vol5MA', 'range_vol10MA',
#        'range_vol25MA', 'oneD_stddev50', 'threeD_stddev50', 'cmf',
#        'close_diff', 'close_diff3', 'close_diff5', 'v_diff_pct', 'adx',
#        'volume_10MA', 'volume_25MA', 'price_10MA', 'price_25MA',
#        'volume_10DDiff', 'volume_25DDiff', 'price_10DDiff', 'price_25DDiff',
#        'day_of_week','close_diff_deviation3','close_diff_deviation','roc_diff','range_vol_diff5',
#        'return_vol_240M', 'volume_vol_240M', 'return_vol_450M', 'volume_vol_450M', 'min_vol_diff', 'min_vol_diff_pct', 'min_volume_vol_diff', 
#        'min_volume_vol_diff_pct', 'return_vol_8H', 'return_vol_16H', 'volume_vol_8H', 'volume_vol_16H', 'hour_vol_diff', 'hour_vol_diff_pct', 'hour_volume_vol_diff', 
#        'hour_volume_vol_diff_pct', 'return_vol_5D', 'return_vol_10D', 'return_vol_30D', 'volume_vol_5D', 'volume_vol_10D', 'volume_vol_30D', 'daily_vol_diff', 'daily_vol_diff_pct',
#        'daily_vol_diff30', 'daily_vol_diff_pct30', 'daily_volume_vol_diff', 'daily_volume_vol_diff_pct', 'daily_volume_vol_diff30', 'daily_volume_vol_diff_pct30']
# testing_features = ['hour','month','day_of_month','price7', 
#        'rsi', 'rsi3', 'rsi5', 'roc', 'roc3', 'roc5', 'threeD_returns_close',
#        'oneD_returns_close', 'range_vol', 'oneD_stddev50', 'threeD_stddev50', 'cmf',
#        'close_diff', 'close_diff3', 'close_diff5', 'v_diff_pct', 'adx',
#        'price_10DDiff', 'price_25DDiff',
#        'day_of_week','close_diff_deviation3','close_diff_deviation','roc_diff','range_vol_diff5',
#         'return_vol_240M', 'volume_vol_240M', 'return_vol_450M', 'volume_vol_450M', 'min_vol_diff', 'min_vol_diff_pct', 'min_volume_vol_diff', 
#        'min_volume_vol_diff_pct', 'return_vol_8H', 'return_vol_16H', 'volume_vol_8H', 'volume_vol_16H', 'hour_vol_diff', 'hour_vol_diff_pct', 'hour_volume_vol_diff', 
#        'hour_volume_vol_diff_pct', 'return_vol_5D', 'return_vol_10D', 'return_vol_30D', 'volume_vol_5D', 'volume_vol_10D', 'volume_vol_30D', 'daily_vol_diff', 'daily_vol_diff_pct',
#        'daily_vol_diff30', 'daily_vol_diff_pct30', 'daily_volume_vol_diff', 'daily_volume_vol_diff_pct', 'daily_volume_vol_diff30', 'daily_volume_vol_diff_pct30', 'return_vol_15M',
#        'volume_vol_15M','return_vol_30M','volume_vol_30M','return_vol_60M','volume_vol_60M','15min_vol_diff_pct','return_vol_3D','volume_vol_3D','return_vol_4H','volume_vol_4H', 'SPY_diff',
#        'SPY_diff3', 'SPY_diff5', 'SPY_1D', 'SPY_3D', 'SPY_5D']

# target_label = 'three_max'
# results_array = []

# i = 0
# while i < number_of_simulations:
#        model_name = f"{title}_temporal_simulation_{i}"
#        print(model_name)
#        random_int = random.randint(24, 40)
#        features = random.sample(testing_features, random_int)
#        print(features)
#        mse_list = run_temporal_simulation(features, target_label, "target_value", dataset_start_date=datetime(2018,1,1,tzinfo=pytz.timezone('US/Eastern')), evaluation_start_date=datetime(2022,10,24,tzinfo=pytz.timezone('US/Eastern')), evaluation_end_date=datetime(2023,10,2,tzinfo=pytz.timezone('US/Eastern')))        
#        mse = sum(mse_list)/len(mse_list)
#        print(mse)
#        print()
#        results_array.append({"model_name": model_name, "features": features, "mse_avg": mse, "num_features": random_int})
#        i += 1

# results_df = pd.DataFrame(results_array)
# results_df.to_csv(f'/Users/charlesmiller/Documents/temporal_simulation_results/{title}/{now.year}_{now.month}_{now.day}.csv', index=False)